This is an extended version of exact_space_time_inference.jl. It combines it with
Optim + ParameterHandling + Zygote to learn the kernel parameters.
If you understand how to use Optim + ParameterHandling + Zygote for an AbstractGP,
e.g. that shown on the README for this package, and how exact_space_time_inference.jl
works, then you should understand this file.

In [1]:
using AbstractGPs
using KernelFunctions
using TemporalGPs

using TemporalGPs: Separable, RectilinearGrid

Load standard packages from the Julia ecosystem

In [2]:
using Optim # Standard optimisation algorithms.
using ParameterHandling # Helper functionality for dealing with model parameters.
using Zygote # Algorithmic Differentiation

Declare model parameters using `ParameterHandling.jl` types.

In [3]:
flat_initial_params, unflatten = ParameterHandling.flatten((
    var_kernel = positive(0.6),
    λ_space = positive(2.5),
    λ_time = positive(2.5),
    var_noise = positive(0.1),
));

Construct a function to unpack flattened parameters and pull out the raw values.

In [4]:
unpack = ParameterHandling.value ∘ unflatten;
params = unpack(flat_initial_params);

function build_gp(params)
    k_space = SEKernel() ∘ ScaleTransform(params.λ_space)
    k_time = Matern52Kernel() ∘ ScaleTransform(params.λ_time)
    k = params.var_kernel * Separable(k_space, k_time)
    return to_sde(GP(k), ArrayStorage(Float64))
end

build_gp (generic function with 1 method)

Construct a rectilinear grid of points in space and time.
Exact inference only works for such grids.
Times must be increasing, points in space can be anywhere.

In [5]:
N = 50;
T = 1_000;
points_in_space = collect(range(-3.0, 3.0; length=N));
points_in_time = RegularSpacing(0.0, 0.01, T);
x = RectilinearGrid(points_in_space, points_in_time);
y = rand(build_gp(params)(x, 1e-4));

Specify an objective function for Optim to minimise in terms of x and y.
We choose the usual negative log marginal likelihood (NLML).

In [6]:
function objective(params)
    f = build_gp(params)
    return -logpdf(f(x, params.var_noise), y)
end

objective (generic function with 1 method)

Optimise using Optim. Takes a little while to compile because Zygote.

In [7]:
training_results = Optim.optimize(
    objective ∘ unpack,
    θ -> only(Zygote.gradient(objective ∘ unpack, θ)),
    flat_initial_params + randn(4), # Add some noise to make learning non-trivial
    BFGS(
        alphaguess = Optim.LineSearches.InitialStatic(scaled=true),
        linesearch = Optim.LineSearches.BackTracking(),
    ),
    Optim.Options(show_trace = true);
    inplace=false,
);

Iter     Function value   Gradient norm 
     0     2.830015e+04     1.887028e+04
 * time: 0.005627155303955078
     1     4.318190e+03     2.229394e+04
 * time: 11.420690059661865
     2    -1.957219e+04     2.328388e+04
 * time: 25.945672035217285
     3    -4.322327e+04     2.325567e+04
 * time: 37.41472911834717
     4    -6.468721e+04     2.345211e+04
 * time: 48.472764015197754
     5    -8.551649e+04     2.330640e+04
 * time: 59.825210094451904
     6    -8.971946e+04     2.313818e+04
 * time: 71.64543414115906
     7    -1.004477e+05     2.265399e+04
 * time: 82.33626317977905
     8    -1.157615e+05     2.139411e+04
 * time: 92.52714610099792
     9    -1.338530e+05     1.779393e+04
 * time: 104.47261214256287
    10    -1.407079e+05     1.506090e+04
 * time: 114.71086120605469
    11    -1.494908e+05     9.890419e+03
 * time: 124.94604110717773
    12    -1.499671e+05     2.041870e+03
 * time: 136.11791610717773
    13    -1.501021e+05     1.055640e+03
 * time: 145.3006300926

Extracting the final values of the parameters.
Should be close to truth.

In [8]:
final_params = unpack(training_results.minimizer)

(var_kernel = 0.6170068785452489, λ_space = 2.4851143171235304, λ_time = 2.4806027128171046, var_noise = 0.00010029789523915114)

Construct the posterior as per usual.

In [9]:
f_post = posterior(build_gp(final_params)(x, final_params.var_noise), y);

Specify some locations at which to make predictions.

In [10]:
T_pr = 1200;
points_in_time_pr = RegularSpacing(0.0, 0.01, T_pr);
x_pr = RectilinearGrid(points_in_space, points_in_time_pr);

Compute the exact posterior marginals at `x_pr`.
This isn't optimised at present, so might take a little while.

In [11]:
f_post_marginals = marginals(f_post(x_pr));
m_post_marginals = mean.(f_post_marginals);
σ_post_marginals = std.(f_post_marginals);

Visualise the posterior marginals. We don't do this during in CI because it causes
problems.

In [12]:
if get(ENV, "TESTING", "FALSE") == "FALSE"
    using Plots
    savefig(
        plot(
            heatmap(reshape(m_post_marginals, N, T_pr)),
            heatmap(reshape(σ_post_marginals, N, T_pr));
            layout=(1, 2),
        ),
        "posterior.png",
    );
end

"/home/runner/work/TemporalGPs.jl/TemporalGPs.jl/docs/src/examples/exact_space_time_learning/posterior.png"

<hr />
<h6>Package and system information</h6>
<details>
<summary>Package information (click to expand)</summary>
<pre>
Status &#96;~/work/TemporalGPs.jl/TemporalGPs.jl/examples/exact_space_time_learning/Project.toml&#96;
  &#91;99985d1d&#93; AbstractGPs v0.5.16
  &#91;ec8451be&#93; KernelFunctions v0.10.53
  &#91;98b081ad&#93; Literate v2.14.0
  &#91;429524aa&#93; Optim v1.7.5
  &#91;2412ca09&#93; ParameterHandling v0.4.6
  &#91;91a5bcdd&#93; Plots v1.38.9
  &#91;e155a3c4&#93; TemporalGPs v0.6.3 &#96;/home/runner/work/TemporalGPs.jl/TemporalGPs.jl#564e73f&#96;
  &#91;e88e6eb3&#93; Zygote v0.6.60
</pre>
To reproduce this notebook's package environment, you can
<a href="./Manifest.toml">
download the full Manifest.toml</a>.
</details>
<details>
<summary>System information (click to expand)</summary>
<pre>
Julia Version 1.8.5
Commit 17cfb8e65ea &#40;2023-01-08 06:45 UTC&#41;
Platform Info:
  OS: Linux &#40;x86_64-linux-gnu&#41;
  CPU: 2 × Intel&#40;R&#41; Xeon&#40;R&#41; CPU E5-2673 v4 @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 &#40;ORCJIT, broadwell&#41;
  Threads: 1 on 2 virtual cores
Environment:
  JULIA_DEBUG &#61; Documenter
  JULIA_LOAD_PATH &#61; :/home/runner/.julia/packages/JuliaGPsDocs/e8FS0/src
</pre>
</details>

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*